<a href="https://colab.research.google.com/github/tanay-98/force-directed-graphs/blob/main/Force_Directed_Graphs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**CS512 Project**

Important links \
1. https://seafile.rlp.net/f/1f06c2943e114d429b48/
2. https://plotly.com/python/animations/ 

In [1]:
import plotly.graph_objects as go
import numpy as np
import math
import time
import random

In [2]:
def f_repel_eades(u, v, P, c_repel):
  """
  Finds force of replusion on u because of v with positions given by P
  """

  if np.array_equal(P[u], P[v]):
    raise Exception(u, v, "are at the same co-ordiantes")
  
  t = np.subtract(P[u], P[v])
  t_norm = math.sqrt(np.dot(t, t.getT()))

  f_repel = (c_repel/(t_norm**3)) * t

  return f_repel

In [3]:
def f_attract_eades(u, v, P, c_repel, c_spring, l):
  """
  Finds force of attraction/replusions on u beause of v with positions given by 
  P
  """

  if np.array_equal(P[u], P[v]):
    raise Exception(u, v, "are at the same co-ordiantes")

  f_repel = f_repel_eades(u, v, P, c_repel)

  t_uv = np.subtract(P[v], P[u])
  t_uv_norm = math.sqrt(np.dot(t_uv, t_uv.getT()))
  t_uv_unit = t_uv / t_uv_norm

  f_spring = c_spring * np.log((t_uv_norm ** 2)/l) * t_uv_unit

  return f_spring - f_repel

In [4]:
def find_max_mag(F_t):
  """
  Finds the maximum magnitude of forces from an array of forces
  """
  max_mag = 0
  for f_key in F_t:
    f = F_t[f_key]
    f_mag = np.dot(f, f.getT())
    if f_mag > max_mag:
      max_mag = f_mag
  
  return math.sqrt(max_mag)

In [5]:
def fd (V, E, P_0, ep, K, c_repel, c_spring, l, alpha=0.1, type="eades"):
  """
  """
  t = 0
  F_t = {}
  F_max = ep + 1
  P = [P_0]

  while t < K and F_max > ep:
    for u in V:

      #Calculating total replusive force on u
      f_repel_total = np.matrix([0.0, 0.0])
      for v in V:
        if v == u:
          continue
        f_repel_total += f_repel_eades(u, v, P[t], c_repel)
      
      #Calculating total attractive force on u
      f_attract_total = np.matrix([0.0, 0.0])
      for w in E[u]:
        if w == u:
          continue
        f_attract_total = f_attract_eades(u, w, P[t], c_repel, c_spring, l)
      
      #Net force on u
      F_t[u] = f_repel_total + f_attract_total

    #Moving each node
    new_pos = {}
    for u in V:
      new_pos[u] = P[t][u] + (alpha/(t+1)) * F_t[u]
    
    P.append(new_pos)
    t+=1
    F_max = find_max_mag(F_t)
  
  print("Successfully ran for ", t, "iteartions")
  return P

Running algorithm on sample input and formatting the output

In [6]:
#Sample A

V_A = [1, 2, 3]

E_A = {1:[2, 3], 2:[1, 3], 3:[1, 2]}

coord_dict = {1: np.matrix([1.0, 1.0]), 
              2: np.matrix([1.05, 2.0]),
              3: np.matrix([1.0, 3.0])
              }

list_of_positions_A = fd(V= V_A, 
         E= E_A,
         P_0= coord_dict,
         ep = 0.1,
         K = 1000,
         c_repel=2,
         c_spring=1,
         l=50,
         alpha=0.99
         )

for pos in list_of_positions_A:
  for v in V_A:
    pos[v] = [pos[v].item(0), pos[v].item(1)]



Successfully ran for  1000 iteartions


In [7]:

#Sample B

V_B = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

E_B = {1:[5, 10], 2:[4, 5, 7], 3:[5, 10], 4:[2, 6, 8], 5:[1, 2, 3], 6:[4, 8], 7:[2, 9], 8:[4, 6], 9:[7, 10], 10:[1, 3, 4, 9]}

coord_dict = {1: np.matrix([random.uniform(0, 8), random.uniform(0, 8)]), 
              2: np.matrix([random.uniform(0, 8), random.uniform(0, 10)]), 
              3: np.matrix([random.uniform(0, 8), random.uniform(0, 8)]),
              4: np.matrix([random.uniform(0, 10), random.uniform(0, 8)]),
              5: np.matrix([random.uniform(0, 8), random.uniform(0, 8)]),
              6: np.matrix([random.uniform(0, 8), random.uniform(0, 8)]),
              7: np.matrix([random.uniform(0, 10), random.uniform(0, 8)]),
              8: np.matrix([random.uniform(0, 8), random.uniform(0, 10)]),
              9: np.matrix([random.uniform(0, 8), random.uniform(0, 8)]),
              10: np.matrix([random.uniform(0, 8), random.uniform(0, 8)])}

list_of_positions_B = fd(V= V_B, 
         E= E_B,
         P_0= coord_dict,
         ep = 0.1,
         K = 1000,
         c_repel=1,
         c_spring=1,
         l=1,
         alpha=0.99
         )

for pos in list_of_positions_B:
  for v in V_B:
    pos[v] = [pos[v].item(0), pos[v].item(1)]

Successfully ran for  1000 iteartions


In [8]:
#Sample C

V_C = [1, 2, 3, 4, 5, 6]

E_C = {1:[2, 6], 2:[1, 3], 3:[6], 4:[5, 6], 5:[4, 6], 6:[1, 3, 4, 5]}

coord_dict = {1: np.matrix([4.0, 6.0]), 
              2: np.matrix([2.0, 7.0]),
              3: np.matrix([4.0, 8.0]),
              4: np.matrix([6.0, 6.0]),
              5: np.matrix([2.5, 7.0]),
              6: np.matrix([5.0, 5.0]),
              }

list_of_positions_C = fd(V= V_C, 
         E= E_C,
         P_0= coord_dict,
         ep = 0.1,
         K = 1000,
         c_repel=1,
         c_spring=1,
         l=5,
         alpha=0.99
         )

for pos in list_of_positions_C:
  for v in V_C:
    pos[v] = [pos[v].item(0), pos[v].item(1)]

Successfully ran for  1000 iteartions


Visualizing the progress of the algorithm

In [9]:
choice = input("Which graph do you want animated? (A or B or C) ")

if choice == 'A':
  list_of_positions = list_of_positions_A
  V = V_A
  E = E_A
  print("Visualizing A")
elif choice == 'B':
  list_of_positions = list_of_positions_B
  V = V_B
  E = E_B
  print("Visualizing B")
else:
  list_of_positions = list_of_positions_C
  V = V_C
  E = E_C
  print("Visualizing C")


data = []
min_x = 100000
min_y = 100000
max_x = 0
max_y = 0 

for pos in list_of_positions:
  node_x = []
  node_y = []
  edge_x = []
  edge_y = []

  for v in V:
    node_x.append(pos[v][0])
    node_y.append(pos[v][1])
    min_x = min(min_x, min(node_x))
    max_x = max(max_x, max(node_x))
    min_y = min(min_y, min(node_y))
    max_y = max(max_y, max(node_y))

    for w in E[v]:
      edge_x.extend([pos[v][0], pos[w][0], "None"])
      edge_y.extend([pos[v][1], pos[w][1], "None"])

  nodes = go.Scatter(x=node_x, y=node_y, mode='markers', marker=dict(size=15))
  edges = go.Scatter(x=edge_x, y=edge_y)
  data.append([edges, nodes])

frames = []
fn = 1
for d in data:
  frames.append(go.Frame(data=d, name=fn))
  fn += 1

print(len(frames)-1, "iterations")

axis_range = dict(
                range=[min(min_x, min_y)-0.5, max(max_x, max_y)+0.5], 
                autorange=False
            )
fig = go.Figure(
    data=data[0],
    layout=go.Layout(
        xaxis=axis_range,
        yaxis=axis_range
    )
    )
fig.update_layout(showlegend=False)
fig.update_layout(title="Original Random Layout")
fig.show()

fig = go.Figure(
    data=data[0],
    layout=go.Layout(
        xaxis=axis_range,
        yaxis=axis_range,
        updatemenus=[
                     dict(
                         type="buttons",
                          buttons=[
                                   dict(
                                       label="Play",
                                        method="animate",
                                        args=[None, {"frame" : {"duration": 1, "redraw": False},
                                                     "fromcurrent": True,
                                                     "transition": {"duration": 4, "easing": "quaratic-in-out"}
                                                     }
                                              ]
                                        ),
                                   dict(
                                       label="Pause",
                                        method="animate",
                                        args=[[None], {"frame" : {"duration": 0, "redraw": False},
                                                     "mode": "immediate",
                                                     "transition": {"duration": 0}
                                                     }
                                              ]
                                   )
                                   ]
                          )
                     ]
                     ),
    frames = frames
    )
fig.update_layout(title="Layout Generated by Algorithm")
fig.update_layout(showlegend=False)
fig.show()

Which graph do you want animated? (A or B or C) A
Visualizing A
1000 iterations


In [10]:
def f(x , y):
  ans = 9*x + 8*y +7
  ans = ans * (2.7182**( (-1)*( x**2 + y**2 )))
  return round( ans, 2 )

In [11]:
x1 = 0.5388
x2 = -0.9277

y1 = 0.5214
y2 = -0.9587

In [12]:
f(x1, y1), f(x1, y2), f(x2, y1), f(x2, y2), 

(9.13, 1.25, 0.91, -1.52)

In [13]:
f(-0.789, -0.701)

-1.87

In [14]:
f(0.354, 0.315)

10.15